Teste

In [1]:
import random

print ("hello, world!")

hello, world!


In [2]:
%pip install --upgrade pip
%pip install python-constraint

from constraint import *
from itertools import combinations


  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip None
error: uninstall-no-record-file

× Cannot uninstall pip None
╰─> The package's contents are unknown: no RECORD file was found for pip.

hint: You might be able to recover from this via: pip install --force-reinstall --no-deps pip==25.2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Instanciar o Problema

In [3]:

days = 'Segunda Terça Quarta Quinta Sexta'.split()
classGroups = 't01 t02 t03'.split()
blocks = 'B01 B02 B03 B04 B05 B06 B07 B08 B09 B10 B11 B12 B13 B14 B15 B16 B17 B18 B19 B20'.split()
lecturers = 'jo mike rob sue'.split()
courses = 'UC11 UC12 UC13 UC14 UC15 UC21 UC22 UC23 UC24 UC25 UC31 UC32 UC33 UC34 UC35'.split()
rooms = 'A C G Lab01 Lab02'.split()


# Constrains

In [4]:

courseClasses = {
    't01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15'],
    't02': ['UC21', 'UC22', 'UC23', 'UC24', 'UC25'],
    't03': ['UC31', 'UC32', 'UC33', 'UC34', 'UC35']
}
classLecturers = {
    'jo' : ['UC11', 'UC21', 'UC22', 'UC31'],
    'mike': ['UC12', 'UC23', 'UC32'],
    'rob': ['UC13', 'UC14', 'UC24', 'UC33'],
    'sue': ['UC15', 'UC25', 'UC34', 'UC35']
}
classRooms = {
    'UC22' : ['Lab01'],
    'UC14' : ['Lab01'],
}

timeRestrictions = {
    'mike' : ['B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20'],
    'rob' : ['B01', 'B02', 'B03', 'B04'],
    'sue' : ['B09', 'B10', 'B11', 'B12', 'B17', 'B18', 'B19', 'B20']
    }



daysBlocks = {
    'Segunda': ['B01', 'B02', 'B03', 'B04'],
    'Terça': ['B05', 'B06', 'B07', 'B08'],
    'Quarta': ['B09', 'B10', 'B11', 'B12'],
    'Quinta': ['B13', 'B14', 'B15', 'B16'],
    'Sexta': ['B17', 'B18', 'B19', 'B20'],
}


onlineLessons = {"UC21_2", "UC31_2"}

physicalRooms = [r for r in rooms if r != "Online"]

# Domínios & Funções auxiliares

In [5]:
# Aulas a agendar
lessonsToSchedule = [f"{courseCode}_{index}" for courseCode in courses for index in (1, 2)]

# Inversões (UC -> docente; UC -> turma)
courseToTeacher = {courseCode: lecturerName for lecturerName, courseList in classLecturers.items() for courseCode in courseList}
courseToGroup = {courseCode: groupName for groupName, courseList in courseClasses.items() for courseCode in courseList}

# ——— Domínio ———
def domainForLesson(lessonName):
    courseCode = lessonName.split("_")[0]
    teacherName = courseToTeacher[courseCode]
    teacherBlockedBlocks = set(timeRestrictions.get(teacherName, []))
    allowedBlocks = [block for block in blocks if block not in teacherBlockedBlocks]

    if lessonName in onlineLessons:
        candidateRooms = ["Online"]                 # aula online não ocupa sala física
    elif courseCode in classRooms and classRooms[courseCode]:
        candidateRooms = classRooms[courseCode]   # salas obrigatórias para a UC
    else:
        candidateRooms = physicalRooms             # qualquer sala física

    return [(block, room) for block in allowedBlocks for room in candidateRooms]


# ——— Funções para ajudar os Constrains ———

def noOverlappingBlock(*lessonAssignments):
    # Cada argumento é um par (block, room) de uma variável da restrição
    blocks = [block for (block, room) in lessonAssignments]
    return len(set(blocks)) == len(blocks)

def notSameRoom(lessonAssignmentLeft, lessonAssignmentRight):
    # Uma aula por (sala, bloco), exceto a aula online
    (blockLeft, roomLeft) = lessonAssignmentLeft
    (blockRight, roomRight) = lessonAssignmentRight
    if roomLeft == "Online" or roomRight == "Online":
        return True
    return not (blockLeft == blockRight and roomLeft == roomRight)

blockToDayMap = {block: day for day, dayBlocks in daysBlocks.items() for block in dayBlocks}
def lessonsOnDiffDays(lessonAssignmentFirst, lessonAssignmentSecond):
    # As 2 aulas da mesma UC em dias diferentes
    return blockToDayMap[lessonAssignmentFirst[0]] != blockToDayMap[lessonAssignmentSecond[0]]

def maxFourDaysLessons(*lessonAssignments):
    # Uma turma deve ter aulas em 4 dias exatamente
    daysWithClasses = {blockToDayMap[block] for (block, room) in lessonAssignments}
    return len(daysWithClasses) == 4

def threeClassesMax(dayBlocks):
    # Máximo de 3 aulas por dia por turma
    dayBlocksSet = set(dayBlocks)

    def isItThreeOrMore(*lessonAssignments):
        count = 0
        for block, room in lessonAssignments:
            if block in dayBlocksSet:
                count += 1
                if count > 3:
                    return False
        return True

    return isItThreeOrMore

def consecutiveClasses(*lessonAssignments):
    # Se uma turma tem aulas num dia, essas aulas devem ser em blocos consecutivos.
    dayToBlocks = {}
    for block, room in lessonAssignments:
        day = blockToDayMap[block]
        dayToBlocks.setdefault(day, []).append(int(block[1:]))

    for day, blockNumbers in dayToBlocks.items():
        if len(blockNumbers) > 1:
            blockNumbers.sort()
            # Verifica se os números dos blocos formam uma sequência contínua
            # [5, 6, 7] -> max(7) - min(5) = 2, que é len(3) - 1. Válido.
            # [5, 7] -> max(7) - min(5) = 2, que é != len(2) - 1. Inválido.
            if max(blockNumbers) - min(blockNumbers) != len(blockNumbers) - 1:
                return False
    return True

def extraRoomsUsed(*lessonAssignments):
      # Contar quantas salas extra cada cadeira está a usar, para além da 1 que é desejável
      physical_rooms = {room for block, room in lessonAssignments if room != "Online"}
      if not physical_rooms:
          return 0
      return max(0, len(physical_rooms) - 1)



# Hard Constrains

In [6]:
def addHardConstrains(problem):
    # 4a) Por docente
    for lecturerName in lecturers:
        variablesForLecturer = [f"{courseCode}_{index}" for courseCode in classLecturers[lecturerName] for index in (1, 2)]
        problem.addConstraint(noOverlappingBlock, tuple(variablesForLecturer))

    # 4b) Por turma
    for groupName in classGroups:
        variablesForGroup = [f"{courseCode}_{index}" for courseCode in courseClasses[groupName] for index in (1, 2)]
        problem.addConstraint(noOverlappingBlock, tuple(variablesForGroup))

    # 4c) Por sala
    for leftLesson, rightLesson in combinations(lessonsToSchedule, 2):
        problem.addConstraint(notSameRoom, (leftLesson, rightLesson))

    #4f) Max 3 aulas por dia por turma
    for groupName in classGroups:
        varsForGroup = [
            lesson for lesson in lessonsToSchedule
            if courseToGroup[lesson.split("_")[0]] == groupName
        ]
        for day, dayBlocks in daysBlocks.items():
            problem.addConstraint(threeClassesMax(dayBlocks), tuple(varsForGroup))



# Soft Constrains

In [7]:
def evaluateSolution(solution):
    penalty = 0
    for groupName in classGroups:
        groupLessons = [
            lesson for lesson in lessonsToSchedule
            if courseToGroup[lesson.split("_")[0]] == groupName
        ]
        assignments = [solution[lesson] for lesson in groupLessons]
        if not consecutiveClasses(*assignments):
            penalty += 10
        if not maxFourDaysLessons(*assignments):
            penalty += 100
    for courseCode in courses:
        assignments = [solution[f"{courseCode}_1"], solution[f"{courseCode}_2"]]
        if not lessonsOnDiffDays(*assignments):
            penalty += 50
        penalty += 20 * extraRoomsUsed(*assignments)
    return penalty


# Descobrir o resultado

In [8]:
bestSolution = None
iterations = 100
minScore = float("inf")

lessonDomains = {lesson: domainForLesson(lesson) for lesson in lessonsToSchedule}
orderedLessons = sorted(lessonsToSchedule, key=lambda lesson: len(lessonDomains[lesson]))

for iteration in range(iterations):
    solver = MinConflictsSolver()
    solver.random = random.Random(iteration)
    ttProblem = Problem(solver)
    for lesson in orderedLessons:
        ttProblem.addVariable(lesson, lessonDomains[lesson])
    addHardConstrains(ttProblem)
    solution = ttProblem.getSolution()
    if not solution:
        continue

    penalty = evaluateSolution(solution)
    if penalty < minScore:
        minScore = penalty
        bestSolution = solution
        print(f"Iteration {iteration + 1} of {iterations}, new best penalty: {minScore}")
    elif (iteration + 1) % 200 == 0:
        if minScore == float("inf"):
            print(f"Iteration {iteration + 1} of {iterations}, no feasible solution yet.")
        else:
            print(f"Iteration {iteration + 1} of {iterations}, best penalty so far: {minScore}")

    if minScore == 0:
        break


Iteration 1 of 100, new best penalty: 470
Iteration 5 of 100, new best penalty: 420
Iteration 7 of 100, new best penalty: 380
Iteration 11 of 100, new best penalty: 330
Iteration 14 of 100, new best penalty: 270
Iteration 53 of 100, new best penalty: 230
Iteration 100 of 100, new best penalty: 180


# Imprimir o resultado


In [9]:
print(f"\nMelhor solução com penalidade {minScore}:\n")

if not bestSolution:
    print("Sem solução encontrada.")
else:
    # agenda dia//bloco
    schedule = {day: {block: [] for block in daysBlocks[day]} for day in days}
    for lesson, (block, room) in bestSolution.items():
        course = lesson.split("_")[0]
        entry = {
            "lesson": lesson,
            "course": course,
            "group": courseToGroup[course],
            "teacher": courseToTeacher[course],
            "room": room,
            "block": block,
            "day": blockToDayMap[block],
        }
        schedule[entry["day"]][block].append(entry)

    # ordenar dia//bloco
    for day in days:
        print(f"\n=== {day} ===")
        for block in daysBlocks[day]:
            entries = schedule[day][block]
            if not entries:
                continue
            print(f"{block}:")
            # ordena por sala e curso para consistência visual
            for e in sorted(entries, key=lambda x: (x["room"], x["course"])):
                print(f"  {e['room']}: {e['course']} ({e['group']}, {e['teacher']})")

    # lista ordenada por bloco (B01..B20)
    print("\n--- Lista por bloco ---")
    ordered = sorted(
        [
            {
                "lesson": lesson,
                "course": lesson.split("_")[0],
                "group": courseToGroup[lesson.split("_")[0]],
                "teacher": courseToTeacher[lesson.split("_")[0]],
                "room": room,
                "block": block,
                "day": blockToDayMap[block],
            }
            for lesson, (block, room) in bestSolution.items()
        ],
        key=lambda e: int(e["block"][1:])
    )
    for e in ordered:
        print(f"{e['block']} ({e['day']}): {e['room']} - {e['course']} ({e['group']}, {e['teacher']})")


Melhor solução com penalidade 180:


=== Segunda ===
B01:
  C: UC34 (t03, sue)
  Lab01: UC22 (t02, jo)
B02:
  Lab01: UC21 (t02, jo)
  Lab02: UC32 (t03, mike)
B03:
  G: UC15 (t01, sue)
  Lab01: UC23 (t02, mike)

=== Terça ===
B05:
  A: UC31 (t03, jo)
  Lab02: UC25 (t02, sue)
B06:
  Lab01: UC14 (t01, rob)
  Lab02: UC34 (t03, sue)
B07:
  Lab01: UC11 (t01, jo)
B08:
  A: UC12 (t01, mike)
  C: UC35 (t03, sue)
  Lab01: UC22 (t02, jo)

=== Quarta ===
B09:
  Lab01: UC14 (t01, rob)
  Online: UC31 (t03, jo)
B10:
  Lab02: UC32 (t03, mike)
B11:
  A: UC23 (t02, mike)
  C: UC13 (t01, rob)
B12:
  G: UC12 (t01, mike)
  Lab01: UC33 (t03, rob)
  Online: UC21 (t02, jo)

=== Quinta ===
B13:
  Lab01: UC24 (t02, rob)
B14:
  Lab01: UC24 (t02, rob)
  Lab02: UC35 (t03, sue)
B15:
  C: UC15 (t01, sue)
B16:
  C: UC25 (t02, sue)
  Lab01: UC33 (t03, rob)

=== Sexta ===
B18:
  Lab01: UC13 (t01, rob)
B20:
  A: UC11 (t01, jo)

--- Lista por bloco ---
B01 (Segunda): Lab01 - UC22 (t02, jo)
B01 (Segunda): C - UC34 (t03, 